# Rain Drops Example - Perpetual Rain Simulation

This example demonstrates a long-running Petri net process where raindrops continuously form, fall, land, and evaporate. The process runs indefinitely until interrupted.

## Process Flow

1. **Form a Drop** → Raindrops are generated from nothing (takes 1 second per drop)
2. **In the Cloud** → Raindrops fall
3. **Falling Down** → Raindrops land in the pool
4. **In the Pool** → Drops may evaporate (1 in 10 chance per drop)

This demonstrates:
- **Perpetual execution**: Drops are continuously generated with no finite input
- **Time delays**: Forming drops takes 1 second
- **Probabilistic behavior**: Random evaporation
- **Long-running processes**: Ideal for testing server-based execution where the process runs independently

## Imports and Setup

In [ ]:
import random
import time
from typing import Optional
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
from pydantic import BaseModel

from petritype.core.executable_graph_components import (
    ExecutableGraph,
    ListPlaceNode,
    FunctionTransitionNode,
    ArgumentEdgeToTransition,
    ReturnedEdgeFromTransition,
    ExecutableGraphOperations,
)
from petritype.core.rustworkx_graph import RustworkxGraph
from petritype.plotting.simple_graphviz import SimpleGraphvizVisualization

## Define Domain Model

We define a simple type:
- `RainDrop`: A raindrop with a unique ID

We'll use a global counter to generate unique IDs for each drop.

In [ ]:
# Global counter for raindrop IDs
_drop_counter = 0


class RainDrop(BaseModel):
    """A raindrop."""
    id: int

## Define Transition Functions

Each transition represents a step in the rain cycle.

Note that `form_a_drop()` has **no parameters** - it generates raindrops from nothing!

In [ ]:
def form_a_drop() -> RainDrop:
    """Create a raindrop from nothing and sleep for 1 second.
    
    This generates raindrops continuously, creating a perpetual process.
    """
    global _drop_counter
    _drop_counter += 1
    print(f"Forming drop {_drop_counter}...")
    time.sleep(1)
    return RainDrop(id=_drop_counter)


def fall(drop: RainDrop) -> RainDrop:
    """Move a raindrop to Falling Down."""
    print(f"Drop {drop.id} is falling...")
    return drop


def land(drop: RainDrop) -> RainDrop:
    """Move a raindrop into the pool."""
    print(f"Drop {drop.id} landed in the pool")
    return drop


def evaporate(drops: list[RainDrop]) -> list[RainDrop]:
    """Check each drop in the pool for evaporation with a 1 in 10 chance per droplet.
    
    Returns the drops that didn't evaporate.
    """
    remaining_drops = []
    evaporated_ids = []
    
    for drop in drops:
        if random.randint(1, 10) != 1:
            # Drop survives - stays in the pool
            remaining_drops.append(drop)
        else:
            # Drop evaporates
            evaporated_ids.append(drop.id)
    
    if evaporated_ids:
        print(f"💨 Drops evaporated: {evaporated_ids}")
    if remaining_drops:
        print(f"💧 Drops remaining in pool: {[d.id for d in remaining_drops]}")
    
    return remaining_drops

## Create the Petri Net Graph

This graph has no initial tokens - the "Form a Drop" transition generates tokens from nothing, creating a perpetual process.

In [ ]:
# Define the graph structure
executable_graph_nodes_and_edges = [
    # Form a Drop transition - creates raindrops from nothing, sleeps 1 second
    # No input edges - this generates tokens continuously!
    FunctionTransitionNode(
        name="Form a Drop",
        function=form_a_drop,
    ),
    ReturnedEdgeFromTransition("Form a Drop", "In the Cloud"),
    
    # In the Cloud place - holds formed raindrops
    ListPlaceNode(name="In the Cloud", type=RainDrop),
    
    # Fall transition - moves raindrop to Falling Down
    ArgumentEdgeToTransition("In the Cloud", "Fall", "drop"),
    FunctionTransitionNode(
        name="Fall",
        function=fall,
    ),
    ReturnedEdgeFromTransition("Fall", "Falling Down"),
    
    # Falling Down place
    ListPlaceNode(name="Falling Down", type=RainDrop),
    
    # Land transition - moves drop into the pool
    ArgumentEdgeToTransition("Falling Down", "Land", "drop"),
    FunctionTransitionNode(
        name="Land",
        function=land,
    ),
    ReturnedEdgeFromTransition("Land", "In the Pool"),
    
    # In the Pool place
    ListPlaceNode(name="In the Pool", type=RainDrop),
    
    # Evaporate transition - removes drops with 1/10 chance each
    ArgumentEdgeToTransition("In the Pool", "Evaporate", "drops"),
    FunctionTransitionNode(
        name="Evaporate",
        function=evaporate,
    ),
    ReturnedEdgeFromTransition("Evaporate", "In the Pool"),
]

## Construct and Visualize the Graph

In [ ]:
executable_graph = ExecutableGraphOperations.construct_graph(executable_graph_nodes_and_edges)
executable_pydigraph = RustworkxGraph.from_executable_graph(executable_graph)
SimpleGraphvizVisualization.graph(executable_pydigraph)

## Execute the Petri Net with Animation

Watch as:
1. Drops continuously form from nothing (1 second each)
2. Drops fall through the cloud
3. Drops land in the pool
4. Some drops evaporate randomly

**Note**: This process runs **indefinitely** - it will never stop on its own! You'll need to interrupt the kernel to stop it.

**Tip**: Limit the execution to a few steps for demonstration purposes.

In [ ]:
# Limit to 20 steps for demonstration (remove this limit to run indefinitely)
max_steps = 20

async for step, diagram, transitions_fired in SimpleGraphvizVisualization.animate_execution_generator(
    executable_graph=executable_graph,
    executable_pydigraph=executable_pydigraph,
):
    clear_output(wait=True)
    print(f"Step {step}")
    display(diagram)
    print(f"Transitions fired: {transitions_fired}")
    
    if step >= max_steps:
        print(f"\\nReached {max_steps} steps - stopping for demonstration.")
        print("This process would normally run indefinitely!")
        break
    
    time.sleep(0.5)
    plt.close()

## Key Observations

- **Perpetual execution**: The "Form a Drop" transition has no inputs, so it can always fire - creating infinite raindrops!
- **Token generation from nothing**: Transitions don't always need input tokens - they can create tokens ex nihilo
- **Timed transitions**: The "Form a Drop" transition sleeps for 1 second, creating a natural rate limit
- **Probabilistic behavior**: The "Evaporate" transition has a 10% chance of removing each drop
- **Token flow**: Drops move through multiple states: Cloud → Falling → Pool → (possibly evaporate)
- **All-tokens pattern**: The "Evaporate" transition takes ALL drops from the pool at once (`list[RainDrop]`)

This pattern is ideal for:
- **Long-running server processes**: Can run indefinitely in the background
- **Event generation**: Creating events/tokens continuously
- **Simulations**: Modeling ongoing processes without predetermined endpoints
- **Testing distributed execution**: Perfect for validating that Petri nets can run independently of observers